In [2]:
import os
import sys
import pickle
import torch
sys.path.append(os.pardir)
from lstm_crf.lstm_crf import BiLSTM_CRF
from lstm_crf.util import parse_text

In [45]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"

with open('../data/word_to_ix_nikkei.pickle', mode='rb') as f:
    word_to_ix = pickle.load(f)
    
tag_to_ix = {'*': 0,
             'サ変接続': 1,
             '一般': 2,
             '人名': 3,
             '副詞可能': 4,
             '助動詞語幹': 5,
             '助数詞': 6,
             '地域': 7,
             '引用': 8,
             '形容動詞語幹': 9,
             '特殊': 10,
             '組織': 11,
             '連語': 12,
             START_TAG: 13,
             STOP_TAG: 14}
ix_to_tag = {i:t for t, i in tag_to_ix.items()}

In [6]:
EMBEDDING_DIM = 100
HIDDEN_DIM = 200
TRAINIG_EPOCH = 5
BATCH_SIZE = 128
# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = 'cpu'

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model.load_state_dict(torch.load('../model/trained_model.model'))

In [29]:
def make_input(text):
    words = parse_text(text)[0]
    #word_id = [word_to_ix[w] for w in words]
    input = []
    result = []
    for w in words:
        try:
            input.append(w)
            result.append(word_to_ix[w])
        except KeyError:
            pass
    return input, torch.tensor(result, dtype=torch.long)

In [30]:
def text_to_id(text):
    text = text.replace('。', ' ')
    text = [word_to_ix[s] for s in list(text)]
    return torch.tensor(text, dtype=torch.long)

In [58]:
new_text = "2019年4月23日、メルカリの決済子会社であるメルペイがオンラインで本人確認できるサービスを開始した。翌24日にはLINE Payも2019年5月初旬に同様のサービスをスタートすると発表した。スマホ決済を利用するうえで必要だが手間のかかる本人確認手続きを簡素に変え、利用者獲得を優位に進める狙いがある。背景にあるのは2018年秋に行われた法改正だ。"

In [59]:
input_word, input_id = make_input(new_text)

result = model.forward(input_id)
for text, label in zip(input_word, result[1]):
    print(f'{text}: {ix_to_tag[label]}')

2019年: *
4月23日: 組織
、: *
メルカリ: *
の: *
決済: 一般
子会社: *
で: *
ある: *
メルペイ: *
が: 一般
オンライン: 一般
で: *
本人確認: *
できる: 一般
サービス: *
を: *
開始: *
し: *
た: *
。: 一般
翌: *
24日: *
に: *
は: 一般
LINE Pay: *
も: *
2019年: *
5月: 一般
初旬: *
に: *
同様: 引用
の: *
サービス: 一般
を: *
スタート: *
する: 一般
と: *
発表: 一般
し: *
た: *
。: 一般
スマホ: *
決済: *
を: *
利用: *
する: *
うえ: *
で: *
必要: 一般
だ: *
が: 一般
手間: *
の: 一般
かかる: *
本人確認: *
手続き: 一般
を: *
簡素: 一般
に: *
変え: 一般
、: *
利用者: *
獲得: *
を: *
優位: *
に: *
進める: 一般
狙い: *
が: *
ある: *
。: *
背景: 一般
に: *
ある: *
の: *
は: 一般
2018年: *
秋: *


In [60]:
a, b = parse_text(new_text)

In [61]:
for aa, bb in zip(a, b):
    print(aa, bb)

2019年 一般
4月23日 一般
、 *
メルカリ 組織
の *
決済 *
子会社 *
で *
ある *
メルペイ *
が 一般
オンライン *
で 一般
本人確認 一般
できる *
サービス *
を 一般
開始 *
し *
た *
。 *
翌 *
24日 一般
に 一般
は *
LINE Pay 一般
も *
2019年 一般
5月 一般
初旬 *
に 一般
同様 *
の *
サービス *
を 一般
スタート *
する *
と 引用
発表 *
し *
た *
。 *
スマホ 一般
決済 *
を 一般
利用 *
する *
うえ 副詞可能
で 一般
必要 *
だ *
が *
手間 *
の 一般
かかる *
本人確認 一般
手続き *
を 一般
簡素 *
に *
変え *
、 *
利用者 一般
獲得 *
を 一般
優位 *
に *
進める *
狙い *
が 一般
ある *
。 *
背景 *
に 一般
ある *
の 一般
は *
2018年 一般
秋 *
に 一般
行わ *
れ *
た *
法改正 一般
だ *
。 *
